# READ ME

This notebook contains all the nessecary code to reproduce the main results for our paper.

## Envoriment
If you do not have the correct envoirment installed, uncomment en run the first cell. If you already have acess to this envoitment, activate it through chancing the kernel

In [2]:
# Create the envoriement from the file
!conda env create -f ./Git_FACT/FairCLIP/fairclip.yml

# Create a new kernel
!python -m ipykernel install --user --name=fairclip --display-name "Python (fairclip)"

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 25.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



tk-8.6.11            | 3.0 MB    | ##################################### | 100% 
wheel-0.37.1         | 33 KB     | ##################################### | 100% 
readline-8.1.2       | 354 KB    | ##################################### | 100% 
libgomp-9.3.0        | 311 KB    | ##################################### | 100% 
tzdata-2025a         | 117 KB    | ##################################### | 100% 
lz4-c-1.9.3          | 185 KB    | ##################################### | 100% 
cryptography-36.0.0  | 1.3 MB    | ##################################### | 100% 
zlib-1.2.11          | 108 KB    | ##################################### | 100% 
libuv-1.40.0         | 736 KB    | ##################################### | 100% 
typing_extensions-4. | 64 KB     | ##########################

done
#
# To activate this environment, use
#
#     $ conda activate fairclip
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Retrieving notices: ...working... done
Installed kernelspec fairclip in /home/lisa/.local/share/jupyter/kernels/fairclip


In [1]:
import os
# Ensure the environment is available as a Jupyter kernel
os.system("python -m ipykernel install --user --name=fairclip --display-name 'Python (fairclip)'")

# Check if the kernel is properly installed
print("Restart the notebook and select the 'Python (fairclip)' kernel from the Kernel menu.")

Installed kernelspec fairclip in /home/lisa/.local/share/jupyter/kernels/fairclip
Restart the notebook and select the 'Python (fairclip)' kernel from the Kernel menu.


In [4]:
import pandas

/home/lisa/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Linear probing

The cell below evaluates the CLIP Vit-14L model, where the linear probing has already been done.

In [2]:
!python -m torch.distributed.launch --master_port=29501 --nproc_per_node=1 ./mae/main_linprobe.py \
        --model_type clip \
        --vl_feats_type image \
        --blip_feats_select avgpool \
        --cfg-path ./LAVIS/lavis/projects/blip2/train/pretrain_stage1.yaml \
        --vision_encoder_weights clip \
        --summary_type gpt-4 \
        --batch_size 512 \
        --model vit_large_patch16 \
        --cls_token \
        --finetune ./tmp_ft_42/best_checkpoint.pth \
        --epochs 1 \
        --blr 0.1 \
        --weight_decay 0.0 \
        --data_path ./FairVLMed/Dataset/dataset-001 \
        --nb_classes 2 \
        --output_dir ./ \
        --log_dir ./ \
        --seed 42 \
        --eval


/home/lisa/anaconda3/envs/fairclip/lib/python3.9/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Traceback (most recent call last):
  File "/home/lisa/Documents/Master/FACT/Git_FACT/FairCLIP/./mae/main_linprobe.py", line 23, in <module>
    from torch.utils.tensorboard import SummaryWriter
  File "/home/lisa/anaconda3/envs/fairclip/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py", line 1, in <module>
    import tensorboard
ModuleNotFoundError: No module named 'tensorboard'
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 0 (pid: 382772) of binar

### Display of results

In [29]:
import pandas as pd
import seaborn as sns
from IPython.display import display


# Load the CSV file
file_path = "best_vit-b16_slo_fundus_eval(1).csv"  # Update this path!
df = pd.read_csv(file_path)

# Clean column names by stripping spaces
df.columns = df.columns.str.strip()

# Extract the first row for baseline metrics
first_row = df.iloc[0]

# Define the correct attribute mappings based on the actual column names
attributes = {
    "Race": {
        "DPD": "dpd_attr0",
        "DEODDs": "eod_attr0",
        "PD": "pdd_head0", #check dit!
        "AUC": "auc",
        "ES-AUC": "esauc_attr0",
        "Group AUC": ["auc_attr0_group0", "auc_attr0_group1", "auc_attr0_group2"],
        "Groups": ["Asian", "Black", "White"]
    },
    "Gender": {
        "DPD": "dpd_attr1",
        "DEODDs": "eod_attr1",
        "AUC": "auc",
        "ES-AUC": "esauc_attr1",
        "Group AUC": ["auc_attr1_group0", "auc_attr1_group1"],
        "Groups": ["Female", "Male"]
    },
    "Ethnicity": {
        "DPD": "dpd_attr2",
        "DEODDs": "eod_attr2",
        "AUC": "auc",
        "ES-AUC": "esauc_attr2",
        "Group AUC": ["auc_attr2_group0", "auc_attr2_group1"],
        "Groups": ["Non-Hispanic", "Hispanic"]
    },
    "Language": {
        "DPD": "dpd_attr3",
        "DEODDs": "eod_attr3",
        "AUC": "auc",
        "ES-AUC": "esauc_attr3",
        "Group AUC": ["auc_attr3_group0", "auc_attr3_group1", "auc_attr3_group2"],
        "Groups": ["English", "Spanish", "Other"]
    }
}

# Create a structured DataFrame for display
table_data = []
for attr, cols in attributes.items():
    row = {
        "Attribute": attr,
        "DPD": first_row[cols["DPD"]],
        "DEOdds": first_row[cols["DEODDs"]],
        "AUC": first_row[cols["AUC"]],
        "ES-AUC": first_row[cols["ES-AUC"]],
    }
    # Add Group-wise AUCs
    for i, col in enumerate(cols["Group AUC"]):
        row[f"{i+1}th Group AUC"] = first_row[col]
    
    table_data.append(row)

# Convert to DataFrame
formatted_df = pd.DataFrame(table_data)
# Display the structured table
display(formatted_df)

table_att = []

for attr, cols in attributes.items():
    row = {
        "Attribute": attr,
        "1st Group Name": cols["Groups"][0] if len(cols["Groups"]) > 0 else None,
        "2nd Group Name": cols["Groups"][1] if len(cols["Groups"]) > 1 else None,
        "3rd Group Name": cols["Groups"][2] if len(cols["Groups"]) > 2 else None
    }
    table_att.append(row)

# Convert to DataFrame
group_names_df = pd.DataFrame(table_att)

# Display the table
display(group_names_df)

,Attribute,DPD,DEOdds,AUC,ES-AUC,1th Group AUC,2th Group AUC,3th Group AUC
0,Race,0.1774,0.1547,0.6378,0.5827,0.6571,0.6884,0.6133
1,Gender,0.0626,0.0885,0.6378,0.6053,0.6151,0.6687,NaN
2,Ethnicity,0.0938,0.0862,0.6378,0.6162,0.6389,0.6038,NaN
3,Language,0.1198,0.1214,0.6378,0.6003,0.6367,0.6051,0.6090


,Attribute,1st Group Name,2nd Group Name,3rd Group Name
0,Race,Asian,Black,White
1,Gender,Female,Male,None
2,Ethnicity,Non-Hispanic,Hispanic,None
3,Language,English,Spanish,Other


     |████████████████████████████████| 294 kB 3.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install --upgrade numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:01:02
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!conda activate fairclip
!pip install --upgrade ipykernel
!python -m ipykernel install --user --name fairclip --display-name "Python (fairclip)"




CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
